In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import PowerTransformer, StandardScaler, MinMaxScaler, OneHotEncoder, LabelEncoder
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
import statsmodels.api as sm


fit the models LinearRegression,Lasso and Ridge and compare the model performances.


In [2]:
df = pd.read_csv('Data_Marketing_Customer_Analysis_Round3.csv')

In [3]:
df.reset_index(inplace=True)

In [4]:
df

,index,region,customer_lifetime_value,response,coverage,education,effective_to_date,month,employment_status,gender,income,location_code,marital_status,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size
0,0,central,4809,no,basic,college,2/18/11,feb,employed,m,48029,suburban,married,61,7,52,0,9,corporate auto,corporate l3,offer3,agent,292,four-door car,medsize
1,1,west region,2228,no,basic,college,1/18/11,jan,unemployed,f,92260,suburban,single,64,3,26,0,1,personal auto,personal l3,offer4,call center,744,four-door car,medsize
2,2,east,14947,no,basic,bachelor,2/10/11,feb,employed,m,22139,suburban,single,100,34,31,0,2,personal auto,personal l3,offer3,call center,480,suv,medsize
3,3,north west,22332,yes,extended,college,1/11/11,jan,employed,m,49078,suburban,single,97,10,3,0,2,corporate auto,corporate l3,offer2,branch,484,four-door car,medsize
4,4,north west,9025,no,premium,bachelor,1/17/11,jan,medical leave,f,23675,suburban,married,117,33,31,0,7,personal auto,personal l2,offer1,branch,707,four-door car,medsize
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10684,10684,central,15563,no,premium,bachelor,1/19/11,jan,unemployed,f,61541,suburban,married,253,12,40,0,7,personal auto,personal l1,offer3,web,1214,luxury car,medsize
10685,10685,north west,5259,no,basic,college,1/6/11,jan,employed,f,61146,urban,married,65,7,68,0,6,personal auto,personal l3,offer2,branch,273,four-door car,medsize
10686,10686,central,23893,no,extended,bachelor,2/6/11,feb,employed,f,39837,rural,married,201,11,63,0,2,corporate auto,corporate l3,offer1,web,381,luxury suv,medsize
10687,10687,west region,11971,no,premium,college,2/13/11,feb,employed,f,64195,urban,divorced,158,0,27,4,6,personal auto,personal l1,offer1,branch,618,suv,medsize


In [6]:
numerical = df.select_dtypes(include = np.number)

In [7]:
X = numerical[['customer_lifetime_value', 'income', 'monthly_premium_auto',
       'months_since_last_claim', 'months_since_policy_inception',
       'number_of_open_complaints', 'number_of_policies']]
y = numerical['total_claim_amount']

Spliting 

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

X_train = pd.DataFrame(X_train, columns=X.columns)
X_test  = pd.DataFrame(X_test, columns=X.columns)

Scaling

In [11]:
scaler = StandardScaler()
scaler.fit(X_train)
scaler.fit(X_test)

X_train_std = scaler.transform(X_train)
X_test_std = scaler.transform(X_test)

In [12]:
X_train.describe()

,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies
count,8551.000000,8551.000000,8551.000000,8551.00000,8551.000000,8551.000000,8551.000000
mean,7994.902701,51817.509063,93.295287,15.13554,48.192960,0.375395,2.983511
std,6848.846659,24717.379264,34.575537,10.13316,27.849503,0.899706,2.398456
min,1898.000000,10074.000000,61.000000,0.00000,0.000000,0.000000,1.000000
25%,4020.500000,29435.000000,68.000000,6.00000,25.000000,0.000000,1.000000
50%,5764.000000,50446.000000,83.000000,14.00000,48.000000,0.000000,2.000000
75%,8964.000000,72194.500000,109.000000,23.00000,71.000000,0.000000,4.000000
max,74228.000000,99981.000000,298.000000,35.00000,99.000000,5.000000,9.000000


#### Apply Linear Regression

In [13]:
from sklearn.linear_model import LinearRegression

In [14]:
lm = LinearRegression() 
model = lm.fit(X_train_std, y_train)

In [15]:
print(model.coef_)
print(model.intercept_)

[-7.59845083e+00 -3.39772375e+01  1.84260338e+02  1.24921898e-01
 -3.04861140e+00  3.02708850e-01  2.41240094e+00]
430.3308761811914


In [25]:
y_pred = model.predict(X_test_std)

In [26]:
y_test = np.array(y_test).ravel()
y_pred = np.array(y_pred).ravel()
compare = pd.DataFrame({'y_test':list(y_test), 'y_pred':list(y_pred)})

In [27]:
from sklearn.metrics import mean_absolute_error as mae, mean_squared_error as mse, r squared 

In [28]:
mse = np.square(np.subtract(y_test,y_pred)).mean()
rmse = np.sqrt(mse)
mae_value = mae(y_test, y_pred)  
r2 = r2_score(y_test, y_pred) 

NameError: name 'r2_score' is not defined

In [ ]:
Applying Lasso

In [37]:
from sklearn.linear_model import Lasso, Ridge

In [38]:
model=Lasso(alpha=0)

model.fit(X_train_std, y_train)
print(f"{model.__class__.__name__}: Train -> {model.score(X_train_std, y_train)}, Test -> {model.score(X_test_std, y_test)}")

Lasso: Train -> 0.411514336844104, Test -> 0.40127309153524027


Applying Ridge

In [39]:
model=Ridge(alpha=0)
model.fit(X_train_std, y_train)
print(f"{model.__class__.__name__}: Train -> {model.score(X_train_std, y_train)}, Test -> {model.score(X_test_std, y_test)}")

Ridge: Train -> 0.411514336844104, Test -> 0.4012730915352398


### Recursive feature elimination
Use feature selection techniques (P-Value, RFE) to select a subset of features to train the model with(if necessary)



In [44]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE  ## recursive feature elemination technique

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

scaler = StandardScaler()
scaler.fit(X_train)
scaler.fit(X_test)

X_train_std = scaler.transform(X_train)
X_test_std = scaler.transform(X_test)

X_train_std = X_train_std.select_dtypes(include=np.number)
X_test_std  = X_test_std.select_dtypes(include=np.number)

X_train_std = pd.DataFrame(X_train_std, columns=X.columns)
X_test_std  = pd.DataFrame(X_test_std, columns=X.columns)

#X_train.isna().sum()
nulls = pd.DataFrame(X_train.isna().sum()).reset_index()
#nulls.head()
nulls.columns = ['Column','nas']
#nulls.head()
#nulls[nulls['nas'] > 0].head()
cols_to_drop = nulls[nulls['nas'] > 0]['Column'] # Too drastic, but made on pourpose for quick filtering (don't do this in production!!)

X_train_std.drop(columns=cols_to_drop, axis=1, inplace = True)
X_test_std.drop(columns=cols_to_drop, axis=1, inplace = True)

#display(X_train)

lm = LinearRegression()

selector = RFE(lm, n_features_to_select= 20, step = 1, verbose = 1) # Step is how many features to add or drop everytime
selector.fit(X_train_std, y_train)

kept_features = selector.get_support(indices = True) #returns an array of integers corresponding to nonremoved features
kept_features = list(X_train_std.iloc[:,kept_features].columns)

X_train_std = selector.transform(X_train_std)
X_test_std  = selector.transform(X_test_std)

X_train_std = pd.DataFrame(X_train_std, columns=kept_features)
X_test_std  = pd.DataFrame(X_test_std, columns=kept_features)

print("Final selected features: ")
display(X_train_std)

AttributeError: 'numpy.ndarray' object has no attribute 'select_dtypes'

Comparing LR, Lasso and Ridge

In [41]:
import pandas as pd
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

def train_and_evaluate_models(models, X, y):
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    for model in models:
        # Fit the model on the training data
        model.fit(X_train, y_train)
        
        # Make predictions on the testing data
        y_pred = model.predict(X_test)
        
        # Calculate the mean squared error (MSE) as a performance metric
        mse = mean_squared_error(y_test, y_pred)
        
        # Print the model's name and MSE
        print(f"Model: {type(model).__name__}")
        print(f"MSE: {mse}\n")

# Assuming 'df' is your DataFrame with features (X) and target variable (y)
# Assuming 'selected_features' is a list of feature names selected through P-value or RFE

# Extract the selected features from the DataFrame
X = df[['customer_lifetime_value', 'income', 'monthly_premium_auto',
       'months_since_last_claim', 'months_since_policy_inception',
       'number_of_open_complaints', 'number_of_policies']]
y = df[['total_claim_amount']]

# Define the models to compare
models = [LinearRegression(), Lasso(), Ridge()]

# Train and evaluate the models
train_and_evaluate_models(models, X, y)


Model: LinearRegression
MSE: 48603.25525355574

Model: Lasso
MSE: 48594.99391672706

Model: Ridge
MSE: 48603.25395271998

